In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
beijing_pm2_5 = fetch_ucirepo(id=381) 
  
# data (as pandas dataframes) 
X = beijing_pm2_5.data.features 
y = beijing_pm2_5.data.targets 
  
# metadata 
print(beijing_pm2_5.metadata) 
  
# variable information 
print(beijing_pm2_5.variables) 

{'uci_id': 381, 'name': 'Beijing PM2.5', 'repository_url': 'https://archive.ics.uci.edu/dataset/381/beijing+pm2+5+data', 'data_url': 'https://archive.ics.uci.edu/static/public/381/data.csv', 'abstract': 'This hourly data set contains the PM2.5 data of US Embassy in Beijing. Meanwhile, meteorological data from Beijing Capital International Airport are also included. ', 'area': 'Climate and Environment', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 43824, 'num_features': 11, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['pm2.5'], 'index_col': ['No'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2015, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C5JS49', 'creators': ['Song Chen'], 'intro_paper': {'title': "Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating", 'authors': 'Xuan Liang, T. Zou, Bin Guo, Shuo Li, Haozhe Zhang,

In [3]:
import datetime

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from IPython.core.debugger import set_trace

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
null_rows = y[y['pm2.5'].isnull()].index.tolist()

In [6]:
x = X.drop(labels=null_rows, axis=0)
y = y.drop(labels=null_rows, axis=0)
x.shape, y.shape

((41757, 11), (41757, 1))

In [7]:
df = x.copy(deep=True)
#df['datetime'] = pd.to_datetime(x[['year', 'month', 'day', 'hour']])
df = df.drop(labels=['year', 'month', 'day', 'hour'], axis=1)
df.shape

(41757, 7)

In [8]:
df_model = pd.get_dummies(df, columns=['cbwd'], dtype=float)

In [9]:
n = len(df_model)

# Split 70:20:10 (train:validation:test)
train_df = df_model[0:int(n*0.7)]
y_train = y[0:int(n*0.7)]

val_df = df_model[int(n*0.7):int(n*0.9)]
y_val = y[int(n*0.7):int(n*0.9)]

test_df = df_model[int(n*0.9):]
y_test = y[int(n*0.9):]

list(map(lambda x: x.shape, [train_df, y_train, val_df, y_val, test_df, y_test]))

[(29229, 10), (29229, 1), (8352, 10), (8352, 1), (4176, 10), (4176, 1)]

In [10]:
scaler = StandardScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

/tmp/ipykernel_34/2621853736.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
/tmp/ipykernel_34/2621853736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
/tmp/ipykernel_34/2621853736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [11]:
def get_data(input_width, label_width, shift, df, target_df, target_col):
    #set_trace()
    x, y = list(), list()
    df = df.reset_index(drop=True)
    target_df = target_df.reset_index(drop=True)
    df = pd.concat([df, target_df], axis=1)
    
    for i in range(0, df.shape[0] - input_width - label_width + 1, shift):
        try:
            features = df.loc[i:i+input_width-1].values
            labels = df.loc[i+input_width:i+input_width+label_width-1, target_col].values

            x.append(features); y.append(labels)
        except Exception as err:
            print(err)
            raise
        
    return x, y

In [12]:
input_width = 96
label_width = 1
shift = 1
target_col = 'pm2.5'

x_train, y_train = get_data(input_width, label_width, shift, train_df, y_train, target_col)
x_val, y_val = get_data(input_width, label_width, shift, val_df, y_val, target_col)
x_test, y_test = get_data(input_width, label_width, shift, test_df, y_test, target_col)

x_train, y_train = torch.tensor(np.array(x_train)), torch.tensor(np.array(y_train))
x_val, y_val= torch.tensor(np.array(x_val)), torch.tensor(np.array(y_val))
x_test, y_test = torch.tensor(np.array(x_test)), torch.tensor(np.array(y_test))

print(list(map(lambda x: x.shape, [x_train, y_train, x_val, y_val, x_test, y_test])))

[torch.Size([29133, 96, 11]), torch.Size([29133, 1]), torch.Size([8256, 96, 11]), torch.Size([8256, 1]), torch.Size([4080, 96, 11]), torch.Size([4080, 1])]


In [13]:
class Embedding(nn.Module):
    def __init__(self, timestep, n_embed):
        super().__init__()
        self.dense1 = nn.Linear(timestep, n_embed) 
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # x: (B, C, T)
        x = self.dense1(x) # (B, C, T) @ (T, n_embed) -> (B, C, n_embed)
        x = self.relu(x)
        
        return x

In [14]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size)
        self.key = nn.Linear(n_embed, head_size)
        self.value = nn.Linear(n_embed, head_size)
        
    def forward(self, x):
        # x : (B, C, D)
        
        query = self.query(x) 
        key = self.key(x)
        value = self.value(x) # (B, C, n_embed) @ (n_embed, head_size) -> (B, C, head_size)
        
        wei = query @ key.transpose(-2, -1) # (B, C, head_size) @ (B, head_size, C) -> (B, C, C)
        wei = F.softmax(wei, dim=-1)
        
        out = wei @ value # (B, C, C) @ (B, C, C) -> (B, C, head_size)
        
        return out

In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, n_embed):
        super().__init__()
        head_size = n_embed // num_heads
        print(f'Creating {num_heads} heads with size {head_size}')
        self.mha = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.mha], dim=-1)
        out = self.proj(out)
        
        return out

In [16]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.linear1 = nn.Linear(n_embed, n_embed)
        self.linear2 = nn.Linear(n_embed, n_embed)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        
        return x

In [17]:
class Block(nn.Module):
    def __init__(self, num_heads, n_embed):
        super().__init__()
        self.mha = MultiHeadAttention(num_heads, n_embed)
        self.ffw = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        x = self.ln1(x + self.mha(x))
        x = self.ln2(x + self.ffw(x))
        
        return x

In [18]:
class Encoder(nn.Module):
    def __init__(self, timestep, n_embed, num_heads, N, output, num_variates):
        super().__init__()
        self.N = N
        self.emb = Embedding(timestep, n_embed)
        self.blocks = nn.ModuleList([Block(num_heads, n_embed) for _ in range(self.N)])
        self.avg1d = nn.AvgPool1d(n_embed) # num_variates
        self.output = nn.Linear(num_variates, output) #n_embed
        
    def forward(self, x):
        B = x.size(0)
        
        x = x.permute(0, 2, 1)
        x = self.emb(x)
        
        for _ in range(self.N):
            x = self.blocks[_](x)
            
        #set_trace()
        #x = self.avg1d(x.permute(0, 2, 1)).view(B, -1) 
        x = self.avg1d(x).view(B, -1)
        x = self.output(x)
        
        return x
        

In [19]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [20]:
training_data = CustomDataset(x_train.float(), y_train.float())
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)

val_data = CustomDataset(x_val.float(), y_val.float())
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=True)

In [21]:
timestep = 96
num_variates = 11
n_embed = 256
num_heads = 4
N = 3
output = 1
learning_rate = 0.0001
num_epochs = 50
criterion = nn.L1Loss()

model = Encoder(timestep, n_embed, num_heads, N, output, num_variates).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Creating 4 heads with size 64
Creating 4 heads with size 64
Creating 4 heads with size 64


In [22]:
def train_one_epoch(model):
    model.train(True)
    running_loss = 0
    
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs) # fix the shape of outputs (32, 11, 4]) -> should be (32, 1)
        #set_trace()
        
        loss_val = criterion(outputs, labels)
        loss_val.backward()
        
        optimizer.step()
        
        running_loss += loss_val.item()
        
    epoch_loss = running_loss / (i + 1)
    
    return epoch_loss

In [23]:
def validation(model):
    model.eval()
    running_loss = 0
    
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            loss_val = criterion(outputs, labels)
            running_loss += loss_val.item()
            
        val_loss = running_loss / (i + 1)
        
    return val_loss

In [ ]:
training_loss, validation_loss = list(), list()

for epoch in range(1, num_epochs+1):
    #set_trace()
    epoch_loss = train_one_epoch(model)
    training_loss.append(epoch_loss)
    
    val_loss = validation(model)
    validation_loss.append(val_loss)
    
    print(f'Epoch {epoch} --> Train: {epoch_loss} and Val: {val_loss}')

Epoch 1 --> Train: 100.36525927731024 and Val: 98.02287180109542
Epoch 2 --> Train: 99.39904496646216 and Val: 96.67081004889437
Epoch 3 --> Train: 97.70598012293995 and Val: 94.71130495293195
Epoch 4 --> Train: 95.52678164116769 and Val: 92.26776435024054
Epoch 5 --> Train: 92.89010216035643 and Val: 89.50990848393403
Epoch 6 --> Train: 90.05980067247879 and Val: 86.67310433794361
Epoch 7 --> Train: 87.20020234310012 and Val: 83.90649947824404
Epoch 8 --> Train: 84.49846964897884 and Val: 81.26753122492354
Epoch 9 --> Train: 81.97136602810002 and Val: 78.71347384489783
Epoch 10 --> Train: 79.59131000908225 and Val: 76.27393624948901
Epoch 11 --> Train: 77.27708609253332 and Val: 73.97830373187398
Epoch 12 --> Train: 75.13283447952354 and Val: 71.83398387228796
Epoch 13 --> Train: 73.1680594778218 and Val: 69.95035106451937
Epoch 14 --> Train: 71.45526600407455 and Val: 68.35282084738562
Epoch 15 --> Train: 69.9924363974813 and Val: 67.04760783587315
Epoch 16 --> Train: 68.830216400447

In [ ]:
# Epoch 50 --> Train: 16.047986085014468 and Val: 14.75692885051402 ::: After permutation
# Epoch 50 --> Train: 16.505132168498704 and Val: 15.850569702858149 ::: No permutation